# Tutorial 6 - Integrating with Suite2p (running Suite2p pipeline and storing results)

In [1]:
import packerlabimaging as pli
from packerlabimaging.processing import suite2p

import numpy as np

No module named 'cellpose'
cannot use anatomical mode, but otherwise suite2p will run normally


In this tutorial, we add the Suite2p submodule to an existing `expobj`. We also show how to run the suite2p pipeline using the `.Suite2p` submodule. Finally, we show how to access and use Suite2p results once the pipeline is run. Note: it is also possible to directly import independently run Suite2p results for this experiment (this is demonstrated in the final section).


Suite2p provides an excellent cell (ROI) detection algorithm for calcium imaging datasets. The pipeline includes registration and spike deconvolution as well. Please familiarize yourself with the basic functionality of Suite2p using their documentation (found [here](https://suite2p.readthedocs.io/en/latest/index.html)) before proceeding with these examples below.

## Adding the Suite2p module

#### Import data analysis object

In [ ]:
# specify the .pkl path to import expobj
prep = 'PS14'
date = '2021-01-28'

expobj: pli.Experiment = pli.import_obj(pkl_path=f'/home/pshah/mnt/qnap/Analysis/{date}/{prep}/{prep}_analysis.pkl')

# set up new Suite2p submodule
expobj.Suite2p = suite2p.Suite2pExperiment(trialsTiffsSuite2p=expobj.Suite2p.tiff_paths_to_use_s2p,
                                          s2pResultsPath=expobj.Suite2p.s2pResultsPath)

Set any necessary `bad_frames` to Suite2p. These frames will be ignored during the ROI detection step. In this example, we set all frames from trials from the `post-4ap` experiment group (`expGroup`) as bad frames. In other experiments, like for example all-optical experiments, we use this feature to set all photostimulation frames as `bad_frames`

In [ ]:
# add Suite2p bad frames from post4ap trials
expobj.Suite2p.bad_frames = []
for trial in expobj.trialIDs:
    if 'post' in expobj.TrialsInformation[trial]['expGroup']:
        trialobj: pli.TwoPhotonImagingTrial = expobj.load_trial(trialID=trial)

        expobj.Suite2p.add_bad_frames(frames=np.arange(trialobj.Suite2p.trial_frames[0], trialobj.Suite2p.trial_frames[1]), bad_frames_npy_loc = expobj.dataPath)

## Prepare for Suite2p run

We next need to setup the Suite2p settings to run for our particular imaging experiment. Use the `.Suite2p.update_ops(dict)` method to update any `ops` options before running to Suite2p.

To get the necessary imaging metadata parameters, here we use one of the `trialobj` contained within the present `expobj` to retrieve that information.

In [ ]:
trialobj: pli.TwoPhotonImagingTrial = expobj.load_trial(trialID=expobj.trialIDs[0])

pix_sz_x = trialobj.imparams.pix_sz_x
pix_sz_y = trialobj.imparams.pix_sz_y
diameter_x = 13 / pix_sz_x
diameter_y = 13 / pix_sz_y
diameter = (int(diameter_x), int(diameter_y)) if int(diameter_y) != int(diameter_x) else int(diameter_x)


Set the new values for Suite2p by using `.Suite2p.update_ops(dict)`

In [ ]:
# new values for ops dictionary
new_ops = {
    'fs': trialobj.imparams.fps,
    'tau': 0.70,
    'num_workers': 50,
    'diameter': diameter,
    'delete_bin': False,  # temporary not deleting binaries in case needed for further testing!
    'batch_size': 2000,  # larger frames will be more RAM intensive, scale user batch size based on num pixels in 512x512 images
    'save_folder': expobj.suite2p_save_path
}

expobj.Suite2p.update_ops(new_ops)
expobj.save()

print('ops:\n', expobj.Suite2p.ops)
print('db:\n', expobj.Suite2p.db)

## Run Suite2p

Now, we are all set to run Suite2p. We have provided the `.Suite2p.s2pRun` method as a wrapper to run the Suite2p on the current `expobj`.

In [ ]:
# run the suite2p pipeline
expobj.Suite2p.s2pRun(expobj=expobj)

## Explore suite2p results - next steps after running Suite2p

TODO

## Import suite2p results directly

If we had run suite2p on our experiment independent of creating the `expobj`, we can add that Suite2p output to the present `expobj` by providing a valid path to the `s2pResultsPath` argument when setting up the `.Suite2p` submodule on our `expobj`.

Note: the output that is required in `s2pResultsPath` is the path to `<../plane0/>` of the Suite2p results output.

In [ ]:
# specify the .pkl path to import expobj
prep = 'PS14'
date = '2021-01-28'

expobj: pli.Experiment = pli.import_obj(pkl_path=f'/home/pshah/mnt/qnap/Analysis/{date}/{prep}/{prep}_analysis.pkl')

# set up new Suite2p submodule
expobj.Suite2p = suite2p.Suite2pExperiment(trialsTiffsSuite2p=expobj.Suite2p.tiff_paths_to_use_s2p,
                                          s2pResultsPath='<path/to/suite2p/output>')